## Общая информация по проекту.

Цель проекта: изучение принципов работы алгоритмов рекомендательных систем, создание прототипа.

Задачи проекта:

- Реализация EDA

- Использование след. моделей:  LightFM, SVD, BaselineOnly(ALS/SGD)

- Создание прототипа

- Получить выводы и дальнейшие рекомендации


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling # import profile for EDA


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are avaHBVCilable in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import surprise
from surprise import Reader, Dataset

from lightfm.cross_validation import random_train_test_split

import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse


from surprise import SVD, AlgoBase, KNNBasic
from surprise import Reader, Dataset
from surprise.model_selection import GridSearchCV

from surprise import BaselineOnly

from surprise import accuracy
from surprise.model_selection import PredefinedKFold



/kaggle/input/recommendationsv4/train.csv
/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/sample_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import json

with open('/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json') as f:
    meta_list = []
    for line in f.readlines():
        meta_list.append(json.loads(line))
        
meta = pd.DataFrame(meta_list)

In [ ]:
meta.info()

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.sample(5)

In [ ]:
train.info()

# EDA

In [ ]:
#using ProfileReport for EDA
profile = pandas_profiling.ProfileReport(train)
profile.to_file("rec_sys.html")
profile

In [ ]:
#train_drop = train.drop(['reviewerName', 'reviewText', 'vote', 'style', 'image','summary','asin','verified'], axis=1) 

In [3]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [4]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))



In [ ]:
train_data.shape, test_data.shape, ratings_coo.shape

# ML

## LightFm

In [ ]:
NUM_THREADS = 10 #число потоков
NUM_COMPONENTS = 41 #число параметров вектора 
NUM_EPOCHS = 14 #число эпох обучения

model = LightFM(learning_rate=0.12, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [ ]:
preds_LFM = model.predict(test_data.userid.values, test_data.itemid.values)




In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds_LFM)


In [ ]:
preds_LFM = model.predict(test.userid.values,
                      test.itemid.values)
normalized_preds = (preds_LFM - preds_LFM.min())/(preds_LFM - preds_LFM.min()).max()
submission['rating']= normalized_preds
submission.to_csv('/kaggle/working/submission_log.csv', index=False)

## SVD

In [5]:
#create matrix for svd/als
reader= Reader(rating_scale=(0,1))
data = Dataset.load_from_df(train_data[['userid','itemid','rating']], reader) 

In [6]:
svd = SVD()

In [7]:
#Hyperparametr tuning
param_grid = {'n_epochs': [5, 20], 
              'lr_all': [0.002, 0.01],
              'reg_all': [0.1, 1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
params = gs.best_params['rmse']

model_svd = SVD(n_epochs=params['n_epochs'],lr_all=params['lr_all'], reg_all=params['reg_all'])

In [8]:
#create model 
trainingSet = data.build_full_trainset()
model = model_svd.fit(trainingSet)

In [11]:
pred_svd = model.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))
est = [i.est for i in pred_svd]
pred_sub = np.array(est)

In [12]:
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)

## BaselineOnly (ALS/SGD)

In [ ]:
bsl_options = {'n_epochs': [5], 'reg_u': [12],'reg_i': [5]}
als = GridSearchCV(KNNBasic, bsl_options, measures=['rmse'], cv=3)
als.fit(data)
#params = als.best_params['rmse']


In [ ]:
# (https://surprise.readthedocs.io/en/stable/prediction_algorithms.html) - documentation

# ALS Algoritm
bsl_options = {'method': 'als',
               'n_epochs': [5,10],
               'reg_u': 15,
               'reg_i': 10
               }
algo = BaselineOnly(bsl_options=bsl_options)

In [ ]:
#learning model
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()
#model_als = algo.fit(trainingSet)


In [ ]:
#pred test for submission
pred_als = model_als.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
#geting submission
est = [i.est for i in pred_als]
pred_sub = np.array(est)
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)


In [ ]:
# SGD Algoritm
bsl_options = {'method': 'sgd',
               'n_epochs': 20,
               'reg': 0.000262055315534682,
               'learning_rate':0.0069826353714177785
               }

algo = BaselineOnly(bsl_options=bsl_options)


In [ ]:
#learning model
trainingSet = data.build_full_trainset()
testSet = trainingSet.build_testset()

algo = BaselineOnly(bsl_options=bsl_options)
model_sgd = algo.fit(trainingSet)

In [ ]:
pred_sgd = model_sgd.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
#geting submission
est = [i.est for i in pred_sgd]
pred_sub = np.array(est)
submission['rating']= pred_sub
submission.to_csv('/kaggle/working/submission_log.csv', index=False)


In [ ]:
#pip install auto-surprise

In [ ]:
# tuning hyperparametrs with auto_surprise
from auto_surprise.engine import Engine
import hyperopt

engine = Engine(['svd', 'svdpp', 'knn_basic', 'knn_baseline'])
# Start the trainer
best_algo, best_params, best_score, tasks = engine.train(
data=data,
target_metric='test_rmse',
cpu_time_limit=60*60*2,
max_evals=100,
hpo_algo=hyperopt.tpe.suggest
)
# Build the model using the best algorithm and hyperparameters
best_model = engine.build_model(best_algo, best_params)

#defined the required parameters
#Best algorithm: baseline_only
#Best hyperparameters: {'bsl_options': {'learning_rate': 0.0069826353714177785, 'method': 'sgd', 'reg': 0.000262055315534682}}



## KKN(CF)

In [ ]:
train_data.itemid.nunique(), train_data.userid.nunique() 

In [ ]:
user_sample = train_data.userid.sample(8000)
item_sample = train_data.itemid.sample(2000)

In [ ]:
from surprise import KNNBasic
from surprise import Reader, Dataset

reader = Reader(rating_scale=(0, 1))
#data = Dataset.load_from_df(train_data[['userid', 'itemid', 'rating']], reader)

data = Dataset.load_from_df(train_data[train_data.userid.isin(user_sample) &
                              train_data.itemid.isin(item_sample)][['userid', 'itemid', 'rating']], reader)

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic()
trainingSet = data.build_full_trainset()
model_knn = knn.fit(trainingSet)


In [ ]:
testSet = trainingSet.build_testset()
predictions = model_knn.test(testSet)

predict = model_knn.test(zip(test.userid.values, test.itemid.values, [0]*len(test.userid.values)))

In [ ]:
est = [i.est for i in predictions]
arr = np.array(est)

In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)

In [ ]:
est = [i.est for i in preds]
arr = np.array(est)

# Выводы

1. EDA
    - Структурное описание датасета приведено в разделе Overview блока EDA
    - Графическое и семантическое описание значений приведены в разделе Variables блока EDA
    - Анализ корреляций приведен в разделе Correlation блока EDA

Заключение по EDA: 
Заметных корреляций не обнаружено(за исключением TimeReview и ItemID)
Тестирование модели на датасете с удаленными признаками (исключая itemid, userid) практически дало схожую оценку(ROC/AUC)

2. ML
Было использовано три модели: LightFM, SVD, ALS/SGD
Наилучший результат показала модель LightFM - 0.75, 
Другие результаты: SVD - 0.734, SGD - 0.737, ALS - 0.729
Модели обучались с подбором гиперпараметров.

3. Протипирование должно показать реальные результаты работы моделей. 

4. Рекомендации. 
Побровать добавить FE. Однако судя по результатам коллег, это не влияет на повышения качества работы модели.
Побровать применить нейроные сети 

